In [5]:
import generator_object, write_run_plot, serial_sim_tools, paths
import numpy as np

from importlib import reload
for obj in [generator_object, write_run_plot, serial_sim_tools]:
    reload(obj)

In [9]:
def remove(name):
    import os, shutil
    path_rem = f'{paths.to_personal_data()}/{name}/images'

    if os.path.isdir(path_rem):
        print(f'\nremoving {path_rem}')
        shutil.rmtree(path_rem)

In [ ]:
import search
reload(search)

search_for_word = 'source laser'

df = search.Generators().word(input = search_for_word, word_slicing = (0,5), print_path = True)
print(df.to_string())


search_inside_chapter = 'Switches and Parameters'
#print(search.Generators().chapter(input = search_inside_chapter))

In [10]:
import generator_object, write_run_plot, serial_sim_tools, animate, plt_file, to_generator_string


from importlib import reload
for obj in [generator_object, to_generator_string, write_run_plot, serial_sim_tools, animate, plt_file]:
    reload(obj)

def lasfoam_tin(n, l):

    gen = generator_object.User_input()
    
    rho, T_ev = 1, 0.5
    z = 12. #
    N0, N1 = 1, 500  # We are operating in 1d wih 500 nodes
    Rmin, Rmax = 0, 30.e-4 #node min max max = 10^-6m = 30 micron 
    ILASER, TLASER = 1e18, 1.e-9 #corresponds to 10^11 W/cm^2
    wavelength_laser = 1 # corresponds to 1*1.06 micrometer

    gen.materials_atom(index = 1, element = "sn")
    gen.materials_atom_isorange('all', n_max = n, lmax = l)
    gen.materials_atom_modeltype(['dca'])
    gen.materials_region([N0,N1], elec_temp = T_ev)
    gen.materials_region_rho(rho)
    gen.materials_region_element(initial_ion_population = 1.0, index = 1)

    gen.geometry(type='slab')
    gen.geometry_nodes(coordinate = "r", scaling_type = "lin", nodes = [N0, N1], nodes_range = [Rmin, Rmax])

    gen.radiation_ebins(n_boundaries = 61, start = 0.1, end = 10**(5))
    gen.radiation_angles(n_rays = 3)

    gen.controls(t_start = 0., t_end = 3e-9, restart = True)

    gen.controls_history(id = 1, value_mutiplier = ILASER, time_multiplier = TLASER)
    gen.controls_history_tv(time = 0., value = 1)
    gen.controls_history_tv(time = 1.0, value = 1.)
    gen.controls_history_tv(time = 1.0, value = 0.)

    gen.sources_source_laser(laser_wavelength = wavelength_laser, option_1 = 'rate', option_2 = 'history', values = [1,1.], nodes = [N0, round(N1/10)])
    gen.parameters(scattering_multiplier = 0,initial_timestep = 10**-13, minimum_timestep = 10**-16, maximum_timestep = 10**-10, time_between_snapshots = 10**(-9))

    gen.popular_switches(continuum_transfer_evolves_temp = True,
                         mesh_treatment = 'staggered centering (node + zones)')
    gen.other_switches(control_calc_thermal_conduct = 'include thermal conduction', 
                       population_calculation = "time dependent diffusion", 
                       temperature_calculation = 'use electronic heating rates for time-dependent')
    
    gen.sources_rswitch(radiation_transfer_algorithm1d= 'do transport using integral formalism', assume_NLTE= True)

    gen.add_plot(title='electron number density', xvars={'time':[],'ir':[0,0]}, yvars={'ne':[0,0]})
    gen.add_plot(title='mass density', xvars={'time':[],'ir':[0,0]}, yvars={'zrho':[0,0]})  
    gen.add_plot(title='charge state', xvars={'time':[],'ir':[0,0]}, yvars={'zbar':[0,0]}) 

    gen.add_plot(title='electron temperature', xvars={'time':[],'ir':[0,0]}, yvars={'tev':[0,0]})
    gen.add_plot(title='radiation remperature', xvars={'time':[],'ir':[0,0]}, yvars={'trv':[0,0]}) 

    gen.add_plot(title='net energy gain due to laser absorption', xvars={'time':[],'ir':[0,0]}, yvars={'heatl':[0,0]})  
    gen.add_plot(title='net energy gain due to radiation absorption', xvars={'time':[],'ir':[0,0]}, yvars={'heatj':[0,0]}) 
    gen.add_plot(title='Net free electron heating rate due to all processes', xvars={'time':[],'ir':[0,0]}, yvars={'heatt':[0,0]}) 

    return gen

import time
times = []
for n in range(5):
    for l in range(n):
        if not (n == 1 and l == 0):

            sim4 = lasfoam_tin(n, l)
            name = f'Howard_scott36_extra_plots/Howard_scott36_{n}_{l}'
            #write_run_plot.write(name = name, object = sim4, longprint=False, plot_duplicates=False)
            #write_run_plot.all(name = name, object = sim4, longprint=False, plot_duplicates=False)
            before  = time.time()
            #write_run_plot.run(name = name, object = sim4, longprint=False)
            #write_run_plot.extra_plot(name = name, make_animation=True)
            #write_run_plot.plot(name = name, object = sim4, longprint=False)
            after = time.time()
            t_minutes = round((after - before)/60, 2)
            print(f'{n} {l} {t_minutes} minutes')
            times.append([n, l , after-before])
        


2 0 0.0 minutes
2 1 0.0 minutes
3 0 0.0 minutes
3 1 0.0 minutes
3 2 0.0 minutes
4 0 0.0 minutes
4 1 0.0 minutes
4 2 0.0 minutes
4 3 0.0 minutes


In [11]:
import generator_object, write_run_plot, serial_sim_tools, animate, plt_file, to_generator_string


from importlib import reload
for obj in [generator_object, to_generator_string, write_run_plot, serial_sim_tools, animate, plt_file]:
    reload(obj)

def spectrum_tin(n, l, T):

    gen = generator_object.User_input()
    
    rho, T_ev = 1, T
    z = 12. #
    N0, N1 = 1, 1  # We are operating in 1d wih 500 nodes
    Rmin, Rmax = 0, 30.e-4 #node min max max = 10^-6m = 30 micron 


    gen.materials_atom(index = 1, element = "sn")
    gen.materials_atom_isorange('all', n_max = n, lmax = l)
    gen.materials_atom_modeltype(['dca'])
    gen.materials_region([N0,N1], elec_temp = T_ev)
    gen.materials_region_rho(rho)
    gen.materials_region_element(initial_ion_population = 1.0, index = 1)

    gen.geometry(type='slab')
    gen.geometry_nodes(coordinate = "r", scaling_type = "lin", nodes = [1, 1], nodes_range = [Rmin, Rmax])

    gen.radiation_ebins(n_boundaries = 201, start = 0.01, end = 2*10**(3))
    gen.radiation_angles(n_rays = 3)

    gen.controls(t_start = 0., t_end = 0., restart = True)

    gen.popular_switches(continuum_transfer_evolves_temp = True,
                         mesh_treatment = 'staggered centering (node + zones)')
    gen.other_switches(control_calc_thermal_conduct = 'include thermal conduction', 
                       population_calculation = "time dependent diffusion", 
                       temperature_calculation = 'use electronic heating rates for time-dependent')
    
    gen.sources_rswitch(radiation_transfer_algorithm1d= 'do transport using integral formalism', assume_NLTE= True)


    gen.add_plot(title='ocupation density for specified element', xvars={'iso':[1]}, 
        yvars={'yisofrac':[1,1]})  
    gen.add_plot(title='EMISSIO',xvars={'energy': []},  # No specific values given, so leaving it empty
        yvars={'cemis': [0, 1]})
    gen.add_plot(title='ABSORPTION',xvars={'energy': []},  # No specific values given, so leaving it empty
        yvars={'ckappa': [0, 1]})
    gen.add_plot(title='photon energies emmision',xvars={'ebins': []},  # No specific values given, so leaving it empty
        yvars={'cemis': [0, 1]})
    gen.add_plot(title='photon energies absorbtion',xvars={'ebins': []},  # No specific values given, so leaving it empty
        yvars={'ckappa': [0, 1] })

    return gen


def temp_loop(n, l):
    trials = []
    name = 'spectral_tin'
    N, M = 4, 4 # this is the multiplot group size and number of groups

    for i in range(M*N):
        tev = round(0.01*2**i,4)

        name_i = f'{name}_{n}_{l}_{tev}'
        #remove(name_i)
        trials.append(name_i)

        gen2 = spectrum_tin(n, l, tev)

        #write_run_plot.write(name = name_i, object = gen2, longprint=False, plot_duplicates=False)
        #write_run_plot.run(name = name_i, object = gen2, longprint=False, plot_duplicates=False)
        #write_run_plot.extra_plot(name = name_i)

        if (i>0 and i%N == 0):

            #serial_sim_tools.plot_all(f'multiplot_{name}_{n}_{l}_{int(i/N)}', trials)
            trials = []

        elif i == M*N -1:
            #serial_sim_tools.plot_all(f'multiplot_{name}_{n}_{l}_{int(i/N)+1}', trials)
            pass

for n in range(6):
    for l in range(n):
        temp_loop(n,l)


In [12]:
import generator_object, write_run_plot, serial_sim_tools, animate, plt_file, to_generator_string


from importlib import reload
for obj in [generator_object, to_generator_string, write_run_plot, serial_sim_tools, animate, plt_file]:
    reload(obj)


blacklist_n_l = [[5,0],[5,2],[5,3],[5,4],[4,3],[4,0],[5,1]]
blacklist_n_l = [[5,4]]

def t_names(t):
    lis = []
    for n in range(4,6):
        for l in range(n):
            if [n,l] not in blacklist_n_l:
                lis.append(f'spectral_tin_{n}_{l}_{t}')
    return lis

for i in range(16):
    tev = round(0.01*2**i,4)
    
    trials = t_names(tev)
    #serial_sim_tools.plot_all(f'multiplot_temp_{tev}', trials)

In [14]:
import generator_object, write_run_plot, serial_sim_tools, animate, plt_file, to_generator_string

from importlib import reload
for obj in [generator_object, to_generator_string, write_run_plot, serial_sim_tools, animate, plt_file]:
    reload(obj)

def lasfoam_tin(n, l):

    gen = generator_object.User_input()
    
    rho, T_ev = 1, 0.05
    z = 12. #
    N0, N1 = 1, 500  # We are operating in 1d wih 500 nodes
    Rmin, Rmax = 0, 30.e-4 #node min max max = 10^-6m = 30 micron 
    ILASER, TLASER = 1e18, 1.e-9 #corresponds to 10^11 W/cm^2
    wavelength_laser = 1 # corresponds to 1*1.06 micrometer

    gen.materials_atom(index = 1, element = "sn")
    gen.materials_atom_isorange('all', n_max = n, lmax = l)
    gen.materials_atom_modeltype(['dca'])
    gen.materials_region([N0,N1], elec_temp = T_ev)
    gen.materials_region_rho(rho)
    gen.materials_region_element(initial_ion_population = 1.0, index = 1)

    gen.geometry(type='slab')
    gen.geometry_nodes(coordinate = "r", scaling_type = "lin", nodes = [N0, N1], nodes_range = [Rmin, Rmax])

    gen.radiation_ebins(n_boundaries = 61, start = 0.1, end = 10**(5))
    gen.radiation_angles(n_rays = 3)

    gen.controls(t_start = 0., t_end = 3e-9, restart = True)

    gen.controls_history(id = 1, value_mutiplier = ILASER, time_multiplier = TLASER)
    gen.controls_history_tv(time = 0., value = 1)
    gen.controls_history_tv(time = 1.0, value = 1.)
    gen.controls_history_tv(time = 1.0, value = 0.)

    gen.sources_source_laser(laser_wavelength = wavelength_laser, option_1 = 'rate', option_2 = 'history', values = [1,1.], nodes = [N0, round(N1/10)])
    gen.parameters(scattering_multiplier = 0,initial_timestep = 10**-13, minimum_timestep = 10**-16, maximum_timestep = 10**-10, time_between_snapshots = 10**(-9))

    gen.popular_switches(continuum_transfer_evolves_temp = True,
                         mesh_treatment = 'staggered centering (node + zones)')
    gen.other_switches(control_calc_thermal_conduct = 'include thermal conduction', 
                       population_calculation = "time dependent diffusion", 
                       temperature_calculation = 'use electronic heating rates for time-dependent')
    
    gen.sources_rswitch(radiation_transfer_algorithm1d= 'do transport using integral formalism', assume_NLTE= True)

    gen.add_plot(title='electron number density', xvars={'time':[],'ir':[0,0]}, yvars={'ne':[0,0]})
    gen.add_plot(title='mass density', xvars={'time':[],'ir':[0,0]}, yvars={'zrho':[0,0]})  
    gen.add_plot(title='charge state', xvars={'time':[],'ir':[0,0]}, yvars={'zbar':[0,0]}) 

    gen.add_plot(title='electron temperature', xvars={'time':[],'ir':[0,0]}, yvars={'tev':[0,0]})
    gen.add_plot(title='radiation remperature', xvars={'time':[],'ir':[0,0]}, yvars={'trv':[0,0]}) 

    gen.add_plot(title='net energy gain due to laser absorption', xvars={'time':[],'ir':[0,0]}, yvars={'heatl':[0,0]})  
    gen.add_plot(title='net energy gain due to radiation absorption', xvars={'time':[],'ir':[0,0]}, yvars={'heatj':[0,0]}) 
    gen.add_plot(title='Net free electron heating rate due to all processes', xvars={'time':[],'ir':[0,0]}, yvars={'heatt':[0,0]}) 

    return gen

import time
times = []
for n in range(6):
    for l in range(n):

        sim4 = lasfoam_tin(n, l)
        name = f'Howard_scott37_{n}_{l}'
        #write_run_plot.write(name = name, object = sim4, longprint=False, plot_duplicates=False)
        #write_run_plot.all(name = name, object = sim4, longprint=False, plot_duplicates=False)
        before  = time.time()
        #write_run_plot.run(name = name, object = sim4, longprint=False)
        after = time.time()
        pr_time = round((after - before)/60,3)
        print(f'{n} {l} {pr_time} minutes')
        times.append([n, l , after-before])
        #write_run_plot.plot_dump(name = name, object = sim4, longprint=False)
        #write_run_plot.extra_plot(name = name, make_animation=True)

1 0 0.0 minutes
2 0 0.0 minutes
2 1 0.0 minutes
3 0 0.0 minutes
3 1 0.0 minutes
3 2 0.0 minutes
4 0 0.0 minutes
4 1 0.0 minutes
4 2 0.0 minutes
4 3 0.0 minutes
5 0 0.0 minutes
5 1 0.0 minutes
5 2 0.0 minutes
5 3 0.0 minutes
5 4 0.0 minutes


In [15]:
# only checking half ev


import generator_object, write_run_plot, serial_sim_tools, animate, plt_file, to_generator_string


from importlib import reload
for obj in [generator_object, to_generator_string, write_run_plot, serial_sim_tools, animate, plt_file]:
    reload(obj)

def spectrum_tin(n, l, T):

    gen = generator_object.User_input()
    
    rho, T_ev = 1, T
    z = 12. #
    N0, N1 = 1, 1  # We are operating in 1d wih 500 nodes
    Rmin, Rmax = 0, 30.e-4 #node min max max = 10^-6m = 30 micron 


    gen.materials_atom(index = 1, element = "sn")
    gen.materials_atom_isorange('all', n_max = n, lmax = l)
    gen.materials_atom_modeltype(['dca'])
    gen.materials_region([N0,N1], elec_temp = T_ev)
    gen.materials_region_rho(rho)
    gen.materials_region_element(initial_ion_population = 1.0, index = 1)

    gen.geometry(type='slab')
    gen.geometry_nodes(coordinate = "r", scaling_type = "lin", nodes = [1, 1], nodes_range = [Rmin, Rmax])

    gen.radiation_ebins(n_boundaries = 201, start = 0.01, end = 2*10**(3))
    gen.radiation_angles(n_rays = 3)

    gen.controls(t_start = 0., t_end = 0., restart = True)

    gen.popular_switches(continuum_transfer_evolves_temp = True,
                         mesh_treatment = 'staggered centering (node + zones)')
    gen.other_switches(control_calc_thermal_conduct = 'include thermal conduction', 
                       population_calculation = "time dependent diffusion", 
                       temperature_calculation = 'use electronic heating rates for time-dependent')
    
    gen.sources_rswitch(radiation_transfer_algorithm1d= 'do transport using integral formalism', assume_NLTE= True)


    gen.add_plot(title='ocupation density for specified element', xvars={'iso':[1]}, 
        yvars={'yisofrac':[1,1]})  
    gen.add_plot(title='EMISSIO',xvars={'energy': []},  # No specific values given, so leaving it empty
        yvars={'cemis': [0, 1]})
    gen.add_plot(title='ABSORPTION',xvars={'energy': []},  # No specific values given, so leaving it empty
        yvars={'ckappa': [0, 1]})
    gen.add_plot(title='photon energies emmision',xvars={'ebins': []},  # No specific values given, so leaving it empty
        yvars={'cemis': [0, 1]})
    gen.add_plot(title='photon energies absorbtion',xvars={'ebins': []},  # No specific values given, so leaving it empty
        yvars={'ckappa': [0, 1] })

    return gen




names = []
for n in range(6):
    for l in range(n):
        name = 'spectrum_tin_halfev'
        tev = .5

        name_i = f'{name}_{n}_{l}'
        names.append(name_i)
        #remove(name_i)
        trials.append(name_i)

        gen2 = spectrum_tin(n, l, tev)

        #write_run_plot.write(name = name_i, object = gen2, longprint=False, plot_duplicates=False)
        #write_run_plot.run(name = name_i, object = gen2, longprint=False, plot_duplicates=False)
        #write_run_plot.extra_plot(name = name_i)

In [17]:
import generator_object, write_run_plot, serial_sim_tools, animate, plt_file, to_generator_string


from importlib import reload
for obj in [generator_object, to_generator_string, write_run_plot, serial_sim_tools, animate, plt_file]:
    reload(obj)

def lasfoam_tin(tev):

    gen = generator_object.User_input()
    
    rho, T_ev = 1, tev
    z = 12. #
    N0, N1 = 1, 500  # We are operating in 1d wih 500 nodes
    Rmin, Rmax = 0, 30.e-4 #node min max max = 10^-6m = 30 micron 
    ILASER, TLASER = 1e18, 1.e-9 #corresponds to 10^11 W/cm^2
    wavelength_laser = 1 # corresponds to 1*1.06 micrometer

    gen.materials_atom(index = 1, element = "sn")
    gen.materials_atom_isorange('all', n_max = 4, lmax = 2)
    gen.materials_atom_modeltype(['dca'])
    gen.materials_region([N0,N1], elec_temp = T_ev)
    gen.materials_region_rho(rho)
    gen.materials_region_element(initial_ion_population = 1.0, index = 1)

    gen.geometry(type='slab')
    gen.geometry_nodes(coordinate = "r", scaling_type = "lin", nodes = [N0, N1], nodes_range = [Rmin, Rmax])

    gen.radiation_ebins(n_boundaries = 61, start = 0.1, end = 10**(5))
    gen.radiation_angles(n_rays = 3)

    gen.controls(t_start = 0., t_end = 3e-9, restart = True)

    gen.controls_history(id = 1, value_mutiplier = ILASER, time_multiplier = TLASER)
    gen.controls_history_tv(time = 0., value = 1)
    gen.controls_history_tv(time = 1.0, value = 1.)
    gen.controls_history_tv(time = 1.0, value = 0.)

    gen.sources_source_laser(laser_wavelength = wavelength_laser, option_1 = 'rate', option_2 = 'history', values = [1,1.], nodes = [N0, round(N1/10)])
    gen.parameters(scattering_multiplier = 0,initial_timestep = 10**-13, minimum_timestep = 10**-16, maximum_timestep = 10**-10, time_between_snapshots = 10**(-9))

    gen.popular_switches(continuum_transfer_evolves_temp = True,
                         mesh_treatment = 'staggered centering (node + zones)')
    gen.other_switches(control_calc_thermal_conduct = 'include thermal conduction', 
                       population_calculation = "time dependent diffusion", 
                       temperature_calculation = 'use electronic heating rates for time-dependent')
    
    gen.sources_rswitch(radiation_transfer_algorithm1d= 'do transport using integral formalism', assume_NLTE= True)

    gen.add_plot(title='electron number density', xvars={'time':[],'ir':[0,0]}, yvars={'ne':[0,0]})
    gen.add_plot(title='mass density', xvars={'time':[],'ir':[0,0]}, yvars={'zrho':[0,0]})  
    gen.add_plot(title='charge state', xvars={'time':[],'ir':[0,0]}, yvars={'zbar':[0,0]}) 

    gen.add_plot(title='electron temperature', xvars={'time':[],'ir':[0,0]}, yvars={'tev':[0,0]})
    gen.add_plot(title='radiation remperature', xvars={'time':[],'ir':[0,0]}, yvars={'trv':[0,0]}) 

    gen.add_plot(title='net energy gain due to laser absorption', xvars={'time':[],'ir':[0,0]}, yvars={'heatl':[0,0]})  
    gen.add_plot(title='net energy gain due to radiation absorption', xvars={'time':[],'ir':[0,0]}, yvars={'heatj':[0,0]}) 
    gen.add_plot(title='Net free electron heating rate due to all processes', xvars={'time':[],'ir':[0,0]}, yvars={'heatt':[0,0]}) 

    return gen

import time
times = []
for t in [0.8, 1.2, 2, 3]:
    sim4 = lasfoam_tin(t)
    name = f'Howard_scott38_{t}'
    #write_run_plot.write(name = name, object = sim4, longprint=False, plot_duplicates=False)
    #write_run_plot.all(name = name, object = sim4, longprint=False, plot_duplicates=False)
    before  = time.time()
    #write_run_plot.run(name = name, object = sim4, longprint=False)
    after = time.time()
    ti = round((after - before)/60, 3)
    print(f'{t} {ti} minutes')
    times.append([n, l , after-before])
    #write_run_plot.plot_dump(name = name, object = sim4, longprint=False)
    #write_run_plot.extra_plot(name = name, make_animation=True)

0.8 0.0 minutes
1.2 0.0 minutes
2 0.0 minutes
3 0.0 minutes


In [25]:
import generator_object, write_run_plot, serial_sim_tools, animate, plt_file, to_generator_string


from importlib import reload
for obj in [generator_object, to_generator_string, write_run_plot, serial_sim_tools, animate, plt_file]:
    reload(obj)

def lasfoam_tin(mult):

    gen = generator_object.User_input()
    
    rho, T_ev = 1, 0.5
    z = 12. #
    N0, N1 = 1, 500  # We are operating in 1d wih 500 nodes
    Rmin, Rmax = 0, 30.e-4 #node min max max = 10^-6m = 30 micron 
    ILASER, TLASER = mult*1e18, 1.e-9 #corresponds to 10^11 W/cm^2
    wavelength_laser = 1 # corresponds to 1*1.06 micrometer

    gen.materials_atom(index = 1, element = "sn")
    gen.materials_atom_isorange('all', n_max = 4, lmax = 2)
    gen.materials_atom_modeltype(['dca'])
    gen.materials_region([N0,N1], elec_temp = T_ev)
    gen.materials_region_rho(rho)
    gen.materials_region_element(initial_ion_population = 1.0, index = 1)

    gen.geometry(type='slab')
    gen.geometry_nodes(coordinate = "r", scaling_type = "lin", nodes = [N0, N1], nodes_range = [Rmin, Rmax])

    gen.radiation_ebins(n_boundaries = 61, start = 0.1, end = 10**(5))
    gen.radiation_angles(n_rays = 3)

    gen.controls(t_start = 0., t_end = 3e-9, restart = True)

    gen.controls_history(id = 1, value_mutiplier = ILASER, time_multiplier = TLASER)
    gen.controls_history_tv(time = 0., value = 1)
    gen.controls_history_tv(time = 1.0, value = 1.)
    gen.controls_history_tv(time = 1.0, value = 0.)

    gen.sources_source_laser(laser_wavelength = wavelength_laser, option_1 = 'rate', option_2 = 'history', values = [1,1.], nodes = [N0, round(N1/10)])
    gen.parameters(scattering_multiplier = 0,initial_timestep = 10**-13, minimum_timestep = 10**-16, maximum_timestep = 10**-10, time_between_snapshots = 10**(-9))

    gen.popular_switches(continuum_transfer_evolves_temp = True,
                         mesh_treatment = 'staggered centering (node + zones)')
    gen.other_switches(control_calc_thermal_conduct = 'include thermal conduction', 
                       population_calculation = "time dependent diffusion", 
                       temperature_calculation = 'use electronic heating rates for time-dependent')
    
    gen.sources_rswitch(radiation_transfer_algorithm1d= 'do transport using integral formalism', assume_NLTE= True)

    gen.add_plot(title='electron number density', xvars={'time':[],'ir':[0,0]}, yvars={'ne':[0,0]})
    gen.add_plot(title='mass density', xvars={'time':[],'ir':[0,0]}, yvars={'zrho':[0,0]})  
    gen.add_plot(title='charge state', xvars={'time':[],'ir':[0,0]}, yvars={'zbar':[0,0]}) 

    gen.add_plot(title='electron temperature', xvars={'time':[],'ir':[0,0]}, yvars={'tev':[0,0]})
    gen.add_plot(title='radiation remperature', xvars={'time':[],'ir':[0,0]}, yvars={'trv':[0,0]}) 

    gen.add_plot(title='net energy gain due to laser absorption', xvars={'time':[],'ir':[0,0]}, yvars={'heatl':[0,0]})  
    gen.add_plot(title='net energy gain due to radiation absorption', xvars={'time':[],'ir':[0,0]}, yvars={'heatj':[0,0]}) 
    gen.add_plot(title='Net free electron heating rate due to all processes', xvars={'time':[],'ir':[0,0]}, yvars={'heatt':[0,0]}) 

    return gen

import time
times = []
for mult in [2, 3, 4.5, 6, 8, 12, 16, 20]:
    sim4 = lasfoam_tin(mult)
    name = f'Howard_scott39_{mult}'
    write_run_plot.write(name = name, object = sim4, longprint=False, plot_duplicates=False)
    #write_run_plot.all(name = name, object = sim4, longprint=False, plot_duplicates=False)
    before  = time.time()
    write_run_plot.run(name = name, object = sim4, longprint=False)
    after = time.time()
    ti = round((after - before)/60, 3)
    print(f'{mult} {ti} minutes')
    times.append([n, l , after-before])
    #write_run_plot.plot_dump(name = name, object = sim4, longprint=False)
    write_run_plot.extra_plot(name = name, make_animation=True)


writing to /home/brewster/Desktop/cretin_package-master/Personal_experiments/Howard_scott39_2/Howard_scott39_2.gen
running cretin with Howard_scott39_2
2 851.867 minutes
adding additional plot to /home/brewster/Desktop/cretin_package-master/Personal_experiments/Howard_scott39_2/images
Created image folder at /home/brewster/Desktop/cretin_package-master/Personal_experiments/Howard_scott39_2/images

writing to /home/brewster/Desktop/cretin_package-master/Personal_experiments/Howard_scott39_3/Howard_scott39_3.gen
running cretin with Howard_scott39_3
3 977.681 minutes
adding additional plot to /home/brewster/Desktop/cretin_package-master/Personal_experiments/Howard_scott39_3/images
Created image folder at /home/brewster/Desktop/cretin_package-master/Personal_experiments/Howard_scott39_3/images

writing to /home/brewster/Desktop/cretin_package-master/Personal_experiments/Howard_scott39_4.5/Howard_scott39_4.5.gen
running cretin with Howard_scott39_4.5
4.5 1103.19 minutes
adding additional p

In [26]:
import generator_object, write_run_plot, serial_sim_tools, animate, plt_file, to_generator_string


from importlib import reload
for obj in [generator_object, to_generator_string, write_run_plot, serial_sim_tools, animate, plt_file]:
    reload(obj)

def lasfoam_tin(mult):

    gen = generator_object.User_input()
    
    rho, T_ev = 1, 0.5
    z = 12. #
    N0, N1 = 1, 500  # We are operating in 1d wih 500 nodes
    Rmin, Rmax = 0, 30.e-4 #node min max max = 10^-6m = 30 micron 
    ILASER, TLASER = mult*1e18, 1.e-9 #corresponds to 10^11 W/cm^2
    wavelength_laser = 1 # corresponds to 1*1.06 micrometer

    gen.materials_atom(index = 1, element = "sn")
    gen.materials_atom_isorange('all', n_max = 4, lmax = 2)
    gen.materials_atom_modeltype(['dca'])
    gen.materials_region([N0,N1], elec_temp = T_ev)
    gen.materials_region_rho(rho)
    gen.materials_region_element(initial_ion_population = 1.0, index = 1)

    gen.geometry(type='slab')
    gen.geometry_nodes(coordinate = "r", scaling_type = "lin", nodes = [N0, N1], nodes_range = [Rmin, Rmax])

    gen.radiation_ebins(n_boundaries = 61, start = 0.1, end = 10**(5))
    gen.radiation_angles(n_rays = 3)

    gen.controls(t_start = 0., t_end = 3e-9, restart = True)

    gen.controls_history(id = 1, value_mutiplier = ILASER, time_multiplier = TLASER)
    gen.controls_history_tv(time = 0., value = 1)
    gen.controls_history_tv(time = 1.0, value = 1.)
    gen.controls_history_tv(time = 1.0, value = 0.)

    gen.sources_source_laser(laser_wavelength = wavelength_laser, option_1 = 'rate', option_2 = 'history', values = [1,1.], nodes = [N0, round(N1/10)])
    gen.parameters(scattering_multiplier = 0,initial_timestep = 10**-13, minimum_timestep = 10**-16, maximum_timestep = 10**-10, time_between_snapshots = 10**(-9))

    gen.popular_switches(continuum_transfer_evolves_temp = True,
                         mesh_treatment = 'staggered centering (node + zones)')
    gen.other_switches(control_calc_thermal_conduct = 'include thermal conduction', 
                       population_calculation = "time dependent diffusion", 
                       temperature_calculation = 'use electronic heating rates for time-dependent')
    
    gen.sources_rswitch(radiation_transfer_algorithm1d= 'do transport using integral formalism', assume_NLTE= True)

    gen.add_plot(title='electron number density', xvars={'time':[],'ir':[0,0]}, yvars={'ne':[0,0]})
    gen.add_plot(title='mass density', xvars={'time':[],'ir':[0,0]}, yvars={'zrho':[0,0]})  
    gen.add_plot(title='charge state', xvars={'time':[],'ir':[0,0]}, yvars={'zbar':[0,0]}) 

    gen.add_plot(title='electron temperature', xvars={'time':[],'ir':[0,0]}, yvars={'tev':[0,0]})
    gen.add_plot(title='radiation remperature', xvars={'time':[],'ir':[0,0]}, yvars={'trv':[0,0]}) 

    gen.add_plot(title='net energy gain due to laser absorption', xvars={'time':[],'ir':[0,0]}, yvars={'heatl':[0,0]})  
    gen.add_plot(title='net energy gain due to radiation absorption', xvars={'time':[],'ir':[0,0]}, yvars={'heatj':[0,0]}) 
    gen.add_plot(title='Net free electron heating rate due to all processes', xvars={'time':[],'ir':[0,0]}, yvars={'heatt':[0,0]}) 

    return gen

import time
times = []
for mult in [0.01, 0.02, 0.05, 0.1, 0.2, 0.4, 50, 80, 100, 150, 200, ]:
    sim4 = lasfoam_tin(mult)
    name = f'Howard_scott39_{mult}'
    write_run_plot.write(name = name, object = sim4, longprint=False, plot_duplicates=False)
    #write_run_plot.all(name = name, object = sim4, longprint=False, plot_duplicates=False)
    before  = time.time()
    write_run_plot.run(name = name, object = sim4, longprint=False)
    after = time.time()
    ti = round((after - before)/60, 3)
    print(f'{mult} {ti} minutes')
    times.append([n, l , after-before])
    #write_run_plot.plot_dump(name = name, object = sim4, longprint=False)
    write_run_plot.extra_plot(name = name, make_animation=True)


writing to /home/brewster/Desktop/cretin_package-master/Personal_experiments/Howard_scott39_0.01/Howard_scott39_0.01.gen
running cretin with Howard_scott39_0.01
0.01 11.536 minutes
adding additional plot to /home/brewster/Desktop/cretin_package-master/Personal_experiments/Howard_scott39_0.01/images
Created image folder at /home/brewster/Desktop/cretin_package-master/Personal_experiments/Howard_scott39_0.01/images

writing to /home/brewster/Desktop/cretin_package-master/Personal_experiments/Howard_scott39_0.02/Howard_scott39_0.02.gen
running cretin with Howard_scott39_0.02
0.02 30.434 minutes
adding additional plot to /home/brewster/Desktop/cretin_package-master/Personal_experiments/Howard_scott39_0.02/images
Created image folder at /home/brewster/Desktop/cretin_package-master/Personal_experiments/Howard_scott39_0.02/images

writing to /home/brewster/Desktop/cretin_package-master/Personal_experiments/Howard_scott39_0.05/Howard_scott39_0.05.gen
running cretin with Howard_scott39_0.05
0.